## 问题
你想通过将你的代码反编译成低级的字节码来查看它底层的工作机制。

## 解决方案
**dis 模块**可以被用来输出任何Python函数的反编译结果。例如：

In [1]:
def countdown(n):
    while n > 0:
        print('T-minus',n)
        n -= 1
    print('Blastoff!')

In [2]:
import dis

In [3]:
dis.dis(countdown)

  2           0 SETUP_LOOP              30 (to 32)
        >>    2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              8 POP_JUMP_IF_FALSE       30

  3          10 LOAD_GLOBAL              0 (print)
             12 LOAD_CONST               2 ('T-minus')
             14 LOAD_FAST                0 (n)
             16 CALL_FUNCTION            2
             18 POP_TOP

  4          20 LOAD_FAST                0 (n)
             22 LOAD_CONST               3 (1)
             24 INPLACE_SUBTRACT
             26 STORE_FAST               0 (n)
             28 JUMP_ABSOLUTE            2
        >>   30 POP_BLOCK

  5     >>   32 LOAD_GLOBAL              0 (print)
             34 LOAD_CONST               4 ('Blastoff!')
             36 CALL_FUNCTION            1
             38 POP_TOP
             40 LOAD_CONST               0 (None)
             42 RETURN_VALUE


## 讨论
当你想要知道你的程序底层的运行机制的时候，dis 模块是很有用的。比如如果你想试着理解性能特征。 被 dis() 函数解析的原始字节码如下所示：

In [4]:
countdown.__code__.co_code

b'x\x1e|\x00d\x01k\x04r\x1et\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00q\x02W\x00t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

如果你想自己解释这段代码，你需要使用一些在 opcode 模块中定义的常量。例如：

In [5]:
c = countdown.__code__.co_code

In [6]:
import opcode

In [7]:
opcode.opname[c[0]]

'SETUP_LOOP'

In [8]:
opcode.opname[c[2]]

'LOAD_FAST'

奇怪的是，在 dis 模块中并没有函数让你以编程方式很容易的来处理字节码。 不过，下面的生成器函数可以将原始字节码序列转换成 opcodes 和参数。

In [9]:
import opcode

def generate_opcodes(codebytes):
    extended_arg = 0
    i = 0
    n = len(codebytes)
    while i < n:
        op = codebytes[i]
        i += 1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
        else:
            oparg = None
        yield (op, oparg)

使用方法如下：

In [10]:
for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)

120 SETUP_LOOP 31774
0 <0> None
100 LOAD_CONST 27393
4 DUP_TOP None
114 POP_JUMP_IF_FALSE 29726
0 <0> None
100 LOAD_CONST 31746
0 <0> None
131 CALL_FUNCTION 258
0 <0> None
124 LOAD_FAST 25600
3 ROT_THREE None
56 INPLACE_SUBTRACT None
0 <0> None
125 STORE_FAST 28928
2 ROT_TWO None
87 POP_BLOCK None
0 <0> None
116 LOAD_GLOBAL 25600
4 DUP_TOP None
131 CALL_FUNCTION 257
0 <0> None
100 LOAD_CONST 21248
0 <0> None


这种方式很少有人知道，你可以利用它替换任何你想要替换的函数的原始字节码。 下面我们用一个示例来演示整个过程

In [11]:
def add(x,y):
    return x + y

In [12]:
c = add.__code__

In [13]:
c

<code object add at 0x0000015D2F8F3270, file "<ipython-input-11-ba4464d9e017>", line 1>

In [14]:
c.co_code

b'|\x00|\x01\x17\x00S\x00'

In [15]:
import types

In [17]:
newbytecode = b'xxxxxxx'

In [18]:
nc = types.CodeType(c.co_argcount, c.co_kwonlyargcount,
...     c.co_nlocals, c.co_stacksize, c.co_flags, newbytecode, c.co_consts,
...     c.co_names, c.co_varnames, c.co_filename, c.co_name,
...     c.co_firstlineno, c.co_lnotab)

In [19]:
nc

<code object add at 0x0000015D3196AD20, file "<ipython-input-11-ba4464d9e017>", line 1>

In [20]:
add.__code__ = nc

In [21]:
add(2,3)

SystemError: unknown opcode

你可以像这样耍大招让解释器奔溃。但是，对于编写更高级优化和元编程工具的程序员来讲， 他们可能真的需要重写字节码。本节最后的部分演示了这个是怎样做到的。你还可以参考另外一个类似的例子： this code on ActiveState